### Imports

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import string
from tqdm.notebook import tqdm
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [9]:
# Create a Kaggle API client
import os
os.environ['KAGGLE_USERNAME'] = 'mustabshiribnamin'
os.environ['KAGGLE_KEY'] = '0eff4183442e547cfe97d8be3e922d42' 
import kaggle as kg
kg.api.dataset_download_files(dataset = "bhavikjikadara/fake-news-detection", path='dataset', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/fake-news-detection


### View Data

In [10]:
true_df = pd.read_csv('dataset/true.csv')
fake_df = pd.read_csv('dataset/fake.csv')

In [11]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [12]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [13]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [14]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


### Data Preprocessing

In [15]:
true_df.duplicated().sum()

206

In [16]:
true_df.drop_duplicates(inplace=True)

In [17]:
fake_df.duplicated().sum()

3

In [18]:
fake_df.drop_duplicates(inplace=True)

In [19]:
true_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [20]:
fake_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [21]:
# labeling the true and false state
true_df['label'] = 0
fake_df['label'] = 1

In [22]:
# concating true and fake data
df = pd.concat([fake_df, true_df], ignore_index=True)
# shuffling the rows of the DataFrame and resetting the index
df = df.sample(frac=1).reset_index(drop=True)

In [23]:
#replace underscores and making title format
df.columns = df.columns.str.replace('_', ' ').str.title()

In [24]:
df.head()

,Title,Text,Subject,Date,Label
0,TREY GOWDY BREAKS DOWN What Clinton Hasn’t Ans...,Trey Gowdy rips into the Clinton case of using...,Government News,"Aug 25, 2016",1
1,EXTORTION? HOW IRAN Used Nuke Deal To Force Ob...,If I were a European and was forced to deal wi...,Government News,"Aug 23, 2016",1
2,EMBARRASSING: Pro-Gun Control Reporter Attempt...,What is it about liberal men that makes a woma...,politics,"Jun 15, 2016",1
3,[VIDEO] EMBOLDENED BY OBAMA’S LAWLESS AMERICA…...,Obama has encouraged this type of behavior wit...,politics,"Sep 3, 2015",1
4,British PM May disagrees with U.S. decision to...,LONDON (Reuters) - British Prime Minister Ther...,worldnews,"December 6, 2017",0


In [25]:
# Clean text
stopword = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean(text):
    text = str(text).lower()
    # remove text within square brackets
    text = re.sub('\[.*?\]', '', text) 
    # remove http links
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # remove html tags
    text = re.sub('<.*?>+', '', text)
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # remove newline chars
    text = re.sub('\n', '', text)
    # remove all word containing numbers
    text = re.sub('\w*\d\w*', '', text)
    # remove stopwords
    text = [word for word in text.split(' ') if word not in stopword]
    # applies stemming to words
    text = [stemmer.stem(word) for word in text]
    text = " ".join(text)
    return text

df['Text'] = df['Text'].apply(lambda x: clean(x))
df['Title'] = df['Title'].apply(lambda x: clean(x))

In [26]:
df.head()

,Title,Text,Subject,Date,Label
0,trey gowdi break clinton hasn’t answer use pri...,trey gowdi rip clinton case use privat email ...,Government News,"Aug 25, 2016",1
1,extort iran use nuke deal forc obama retreat e...,european forc deal massiv influx muslim refuge...,Government News,"Aug 23, 2016",1
2,embarrass progun control report attempt hit jo...,liber men make woman want shoot testosteron ma...,politics,"Jun 15, 2016",1
3,embolden obama’ lawless america…black suprema...,obama encourag type behavior divis rhetor prov...,politics,"Sep 3, 2015",1
4,british pm may disagre us decis move embassi j...,london reuter british prime minist theresa ma...,worldnews,"December 6, 2017",0


### Sentiment Analysis

In [27]:
sia = SentimentIntensityAnalyzer()
result = {}

for i, row in tqdm(df.iterrows(), total=len(df), desc="Sentiment Analysis"):
    text = row['Text']
    my_id = row['Title'] 
    result[my_id] = sia.polarity_scores(text)

Sentiment Analysis:   0%|          | 0/44689 [00:00<?, ?it/s]

In [28]:
vaders = pd.DataFrame(result).T
vaders = vaders.reset_index().rename(columns={'index': 'Title'})
vaders = vaders.merge(df, how='left')

In [29]:
vaders.head()

,Title,neg,neu,pos,compound,Text,Subject,Date,Label
0,trey gowdi break clinton hasn’t answer use pri...,0.148,0.852,0.000,-0.5106,trey gowdi rip clinton case use privat email ...,Government News,"Aug 25, 2016",1
1,trey gowdi break clinton hasn’t answer use pri...,0.148,0.852,0.000,-0.5106,trey gowdi rip clinton case use privat email ...,politics,"Aug 25, 2016",1
2,extort iran use nuke deal forc obama retreat e...,0.187,0.813,0.000,-0.8934,european forc deal massiv influx muslim refuge...,Government News,"Aug 23, 2016",1
3,extort iran use nuke deal forc obama retreat e...,0.187,0.813,0.000,-0.8934,european forc deal massiv influx muslim refuge...,left-news,"Aug 23, 2016",1
4,embarrass progun control report attempt hit jo...,0.184,0.711,0.106,-0.9674,liber men make woman want shoot testosteron ma...,politics,"Jun 15, 2016",1


### Model Training

In [30]:
from imblearn.over_sampling import RandomOverSampler
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix


In [31]:
df1 = vaders.copy()

In [32]:
df1['content'] = df1['Title'] + ' ' + df1['Text']

In [33]:
X = df1['content']
y = df1['Label']

In [35]:
#Scaling "Label" oversampling for balanced class distribution
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(df1['content'].values.reshape(-1, 1), df1['Label'])
X_res = X_res.flatten()

In [36]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='OOV')
tokenizer.fit_on_texts(X_res)
sequences = tokenizer.texts_to_sequences(X_res)
padded = pad_sequences(sequences, maxlen=1000)

In [37]:
# Save the tokenizer
with open('Models/tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

In [31]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded, y_res, test_size=0.2, random_state=42)

In [32]:
model = keras.Sequential()

# converts integer-encoded word indices (from the tokenized text) into dense vectors of fixed size
model.add(keras.layers.Embedding(input_dim=10000, output_dim=32))  
model.add(keras.layers.SimpleRNN(32, activation='relu', dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model_checkpoint = keras.callbacks.ModelCheckpoint('model.{epoch:02d}-{val_loss:.2f}.keras', monitor='val_loss', save_best_only=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 439s 370ms/step - accuracy: 0.8400 - loss: 57935096.0000 - val_accuracy: 0.9255 - val_loss: 0.1983
Epoch 2/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 385s 328ms/step - accuracy: 0.9441 - loss: 0.1528 - val_accuracy: 0.9382 - val_loss: 0.1588
Epoch 3/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 351s 299ms/step - accuracy: 0.9588 - loss: 0.1097 - val_accuracy: 0.9326 - val_loss: 0.1643
Epoch 4/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 495s 422ms/step - accuracy: 0.9610 - loss: 0.1447 - val_accuracy: 0.9391 - val_loss: 0.1560
Epoch 5/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 499s 425ms/step - accuracy: 0.9699 - loss: 0.0816 - val_accuracy: 0.9415 - val_loss: 0.1545
Epoch 6/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 531s 450ms/step - accuracy: 0.9734 - loss: 0.0745 - val_accuracy: 0.9392 - val_loss: 0.1582
Epoch 7/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 470s 400ms/step - accuracy: 0.9755 - loss: 0.0674 - val_accuracy: 0.9425 - val_loss: 0.1505
Epoch 8/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 504s 

In [34]:
evaluation_result = model.evaluate(X_test, y_test)

print("Test Loss:", evaluation_result[0])
print("Test Accuracy:", evaluation_result[1])

294/294 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - accuracy: 0.9491 - loss: 0.1469
Test Loss: 0.14805002510547638
Test Accuracy: 0.9451661109924316


In [35]:
model.summary()
model.save('simple_rnn_model.keras')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1000, 32)       │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 966,341 (3.69 MB)

 Trainable params: 322,113 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 644,228 (2.46 MB)

In [36]:
model_lstm = keras.Sequential()
model_lstm.add(keras.layers.Embedding(input_dim=10000, output_dim=32))
model_lstm.add(keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True)))
model_lstm.add(keras.layers.Dropout(0.5))
model_lstm.add(keras.layers.LSTM(32))
model_lstm.add(keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.01)))
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_lstm.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model_lstm_checkpoint = keras.callbacks.ModelCheckpoint('model_lstm.{epoch:02d}-{val_loss:.2f}.keras', monitor='val_loss', save_best_only=True)

history = model_lstm.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 1615s 1s/step - accuracy: 0.9091 - loss: 0.2063 - val_accuracy: 0.9952 - val_loss: 0.0385
Epoch 2/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 1614s 1s/step - accuracy: 0.9938 - loss: 0.0376 - val_accuracy: 0.9941 - val_loss: 0.0382
Epoch 3/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 1684s 1s/step - accuracy: 0.9961 - loss: 0.0313 - val_accuracy: 0.9963 - val_loss: 0.0279
Epoch 4/100
1174/1174 ━━━━━━━━━━━━━━━━━━━━ 1393s 1s/step - accuracy: 0.9986 - loss: 0.0227 - val_accuracy: 0.9983 - val_loss: 0.0230
Epoch 5/100
 914/1174 ━━━━━━━━━━━━━━━━━━━━ 8:01 2s/step - accuracy: 0.9990 - loss: 0.0211

In [ ]:
evaluation_result = model_lstm.evaluate(X_test, y_test)

print("Test Loss:", evaluation_result[0])
print("Test Accuracy:", evaluation_result[1])

: 

In [ ]:
model_lstm.summary()
model_lstm.save('lstm_model.keras')

: 

In [ ]:
new_model = keras.models.load_model('lstm_model.keras')
predictions = new_model.predict(X_test)

: 